I want to generate logs to see what's going on

In regular python, I could just add print() calls and the logs would be a side effect of the function. 

With a pure function, there is only one choice: we must return the log message as part of the function's return value
```python
def f(x):
    return (x*x, "log Message f : x -> x*x")
```
This tuple will include the log value

In [5]:
# logger: pass around a tuple of (value, log)
def f(x):
    value, log = x
    return (value*value, 
            log + f"I squared {value}\n")
def g(x):
    value, log = x
    return (2*value, log)

def h(x):
    value, log = x
    return (1 + value,
            log + f"I added 1 to {value}\n")

v0 = (4, "")
v1 = f(v0)
v2 = g(v1)
v3 = h(v2)
#or: v3 = h(g(f(v0)))
print(f"Answer: {v3[0]}")
print(f"Log: {v3[1]}")



Answer: 33
Log: I squared 4
I added 1 to 32



In [11]:
# Let's refactor into a class to make it more pythonic

class ValueAndLog:
    def __init__(self, value, log):
        self.value = value
        self.log = log
    
    

def f(data):
    return_value = data.value * data.value
    return_log = data.log  + f"I squared {data.value}\n"
    return ValueAndLog(data.value * data.value, 
                        data.log  + f"I squared {data.value}\n")

def g(data):
    return ValueAndLog(2*data.value, data.log)

def h(data):
    return ValueAndLog(1+data.value, 
                        data.log + f"I added 1 to {data.value}\n")

v0 = ValueAndLog(4, "")
v1 = f(v0)
v2 = g(v1)
v3 = h(v2)
#or: v3 = h(g(f(v0)))

print("Answer:", v3.value)
print("Log:", v3.log)

Answer: 33
Log: I squared 4
I added 1 to 32



In [24]:
# Now let's refactor to remove the the duplicate parts

class ValueAndLog:
    def __init__(self, value, log):
        self.value = value
        self.log = log
    
    def __rshift__(self, func):
        result = func(self.value)
        return ValueAndLog(result.value,
                            self.log + result.log)
    @staticmethod
    def unit(v):
        return ValueAndLog(v,"")

    @staticmethod
    def lift(func):
        return lambda val: ValueAndLog(func(val), "")

def f(x):
    return ValueAndLog(x*x, f"I squared {x}\n")

def g(x):
    return ValueAndLog(2*x, "")

def h(x):
    return ValueAndLog(1+x,  f"I added 1 to {x}\n")

v3 = ValueAndLog.unit(3) >> f >> g >> h

print("Answer:", v3.value)
print("Log:", v3.log)

Answer: 19
Log: I squared 3
I added 1 to 18



In [31]:
class ValueAndLog:
    def __init__(self, value, log):
        self.value = value
        self.log = log

    def __rshift__(self, func):
        result = func(self.value)
        return ValueAndLog(result.value, self.log + result.log)

    @staticmethod
    def unit(v):
        return ValueAndLog(v, "")

    @staticmethod
    def lift(f):
        return compose(ValueAndLog.unit, f)

def compose(f1, f2):
    return lambda v: f1(f2(v))

def f(x):
    return ValueAndLog(x*x, "I squared %d\n" % x)

def g(x):
    return 2*x

def h(x):
    return ValueAndLog(1+x, "I added 1 to %d\n" % x)

v3 = ValueAndLog.unit(3) >> f >> ValueAndLog.lift(g) >> h
print("Answer:", v3.value)
print("Log:", v3.log)

Answer: 19
Log: I squared 3
I added 1 to 18



this is an example of a more general composition pattern which consists of:<br>
* A type (class) which wraps some data<br>
* A “unit” function, which wraps a plain value inside an instance of this type
* A “bind” function, which selectively applies another function to the unwrapped value, and manipulates that function’s return value

It turns out there are a whole bunch of other ways to use this pattern, each with its own (class + unit + bind(>>)). You could almost say that they form a “category” of their own. <br><br>Examples include:
* Dealing with nil or missing values: If the input value is not nil, then the bind method calls the provided function. If the input value is nil, then the bind method skips it entirely, and returns nil.
* Dealing with errors: The return value from a function can be either a value or an error message. If the value passed along the chain to bind is an error, then the error propagates directly to the output.
* Managing shared state: A state value is passed along the chain, and each function says how it wants to update that state